In [3]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# Veri setlerini okuyun
df_customers = pd.read_csv('data/olist_customers_dataset.csv')
df_order_items = pd.read_csv('data/olist_order_items_dataset.csv')
df_orders = pd.read_csv('data/olist_orders_dataset.csv')
df_products = pd.read_csv('data/olist_products_dataset.csv')

# Gerekli veri birleştirmelerini yapın
df_customers_orders = df_customers.merge(df_orders, on='customer_id')
orders_order_items = df_orders.merge(df_order_items, on='order_id')

# Üst müşteri ID'lerini bulun
top_customer_ids = orders_order_items['customer_id'].value_counts().head(10).index

# Tüm müşteriler için önerileri alacak bir fonksiyon oluşturun
def get_all_recommendations(cosine_sim, orders_data, top_customer_ids):
    all_recommendations = {}
    for customer_id in top_customer_ids:
        recommendations = get_recommendations(customer_id, cosine_sim, orders_data)
        all_recommendations[customer_id] = recommendations
    return all_recommendations

# Öneri sistemi oluşturmak için gerekli işlemleri gerçekleştirin
def create_recommendations(df_orders, df_order_items, top_customer_ids):
    # Müşteri ve ürün verilerini birleştirin
    orders_order_items = df_orders.merge(df_order_items, on='order_id')
    
    # Üst müşteri ID'lerini filtreleyin
    top_customer_data = orders_order_items[orders_order_items['customer_id'].isin(top_customer_ids)]
    
    # Ürünleri birleştirin
    orders_data = top_customer_data.groupby('customer_id')['product_id'].apply(list).reset_index()
    
    # TfidfVectorizer kullanarak ürün özellik vektörlerini oluşturun
    vectorizer = TfidfVectorizer(tokenizer=lambda x: x, lowercase=False)
    orders_data['product_id_str'] = orders_data['product_id'].apply(lambda x: ' '.join(x))
    tfidf_matrix = vectorizer.fit_transform(orders_data['product_id_str'])
    
    # Cosine similarity kullanarak müşteriler arasındaki benzerliği hesaplayın
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    
    # Tüm müşteriler için önerileri alın
    all_recommendations = get_all_recommendations(cosine_sim, orders_data, top_customer_ids)
    
    return all_recommendations

# Müşteriye öneri yapmak için bir fonksiyon tanımlayın
def get_recommendations(customer_id, cosine_sim, orders_data):
    if customer_id in orders_data['customer_id'].values:
        idx = orders_data[orders_data['customer_id'] == customer_id].index[0]
        
        # Cosine benzerlik skorunu alın
        sim_scores = list(enumerate(cosine_sim[idx]))
        
        # Benzerlik skorlarına göre sıralama yapın
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        
        # Kendi dışındaki en benzer 5 müşteriyi alın
        similar_customers_indices = [i[0] for i in sim_scores[1:6]]
        
        # Benzer müşterilerin aldığı ürünleri önerin
        recommendations = orders_data.iloc[similar_customers_indices]['product_id']
        
        return recommendations
    else:
        return "Müşteri bulunamadı."

# Önerileri oluşturun
all_recommendations = create_recommendations(df_orders, df_order_items, top_customer_ids)

# Önerileri gösterin
for customer_id, recommendations in all_recommendations.items():
    print("Müşteri ID:", customer_id)
    print("Önerilen Ürünler:")
    print(recommendations)
    print()


Müşteri ID: fc3d1daec319d62d49bfb5e1f83123e9
Önerilen Ürünler:
2    [d34c07a2d817ac73f4caf8c574215fed, d34c07a2d81...
8    [e11092e9722d1b6f8c18cd8947a1daff, b6ada738186...
1    [710b7c26b7a742f497bba45fab91a25f, a9d9db064d4...
0    [89b190a046022486c635022524a974a8, 89b190a0460...
4    [37eb69aca8718e843d897aa7b82f462d, 37eb69aca87...
Name: product_id, dtype: object

Müşteri ID: bd5d39761aa56689a265d95d8d32b8be
Önerilen Ürünler:
8    [e11092e9722d1b6f8c18cd8947a1daff, b6ada738186...
1    [710b7c26b7a742f497bba45fab91a25f, a9d9db064d4...
7    [422879e10f46682990de24d770e7f83d, 422879e10f4...
9    [270516a3f41dc035aa87d220228f844c, 05b515fdc76...
2    [d34c07a2d817ac73f4caf8c574215fed, d34c07a2d81...
Name: product_id, dtype: object

Müşteri ID: be1b70680b9f9694d8c70f41fa3dc92b
Önerilen Ürünler:
7    [422879e10f46682990de24d770e7f83d, 422879e10f4...
4    [37eb69aca8718e843d897aa7b82f462d, 37eb69aca87...
3    [44a5d24dd383324a421569ca697b13c2, 44a5d24dd38...
8    [e11092e9722d1b6f8c18cd89

C:\Users\danya\anaconda3\envs\test\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
